## Import Bibliotek

In [106]:
import numpy as np

## Wczytanie danych

In [ ]:
nazwa_pliku = 'input.txt'

# Funkcja, która zastępuje przecinki jako separator dziesiętny na kropki - w celu poprawnego wczytania danych za
# pomocą np.loadtxt
def replace_commas_with_periods(filename):
    with open(filename, 'r') as file:
        contents = file.read()
        new_contents = contents.replace(',', '.')
    with open(filename, 'w') as file:
        file.write(new_contents)

replace_commas_with_periods(nazwa_pliku)
coords = np.loadtxt('./' + nazwa_pliku, delimiter=' ')
print("Dane wejściowe:")
print(coords)

## Przypisanie danych do zmiennych

In [ ]:
xi = coords[:, 0]
fx = coords[:, 1]

## Funkcja która oblicza wartość następnej kolumny na podstawie danych z poprzedniej

In [109]:
def get_next_column(fx: np.ndarray):
   next_column = np.array([])
   last_f = fx[-1]
   for index, f in np.ndenumerate(fx):
    if f != last_f:
        next_column = np.append(next_column, round((fx[index[0]+1] - f), 2))
   return next_column
print("Sprawdzenie poprawności działania funkcji:")
print(get_next_column(fx))

Sprawdzenie poprawności działania funkcji:
[ 2.5  -1.35 -3.85 -5.   -4.8  -3.25]


## Funkcja która oblicza kolejne kolumny w tabeli

In [110]:
columns = []

def calculate_cols(column: np.ndarray):
    global columns
    col = get_next_column(column)
    columns.append(col)
    if len(np.unique(col)) != 1:
        calculate_cols(col)
    else:
        return

calculate_cols(fx)

## Wyświetlenie wyników

In [111]:
for i in range(len(columns)):
    print(f'Column {i+1}: {columns[i]}')

Column 1: [ 2.5  -1.35 -3.85 -5.   -4.8  -3.25]
Column 2: [-3.85 -2.5  -1.15  0.2   1.55]
Column 3: [1.35 1.35 1.35 1.35]


In [112]:
for column in columns:
    print(column[3])

-5.0
0.2
1.35


# Metoda róznicy centralnej

In [118]:
print(fx)
for index, number in enumerate(fx):
    fx.insert(index, None)
print(fx)

def get_next_column(fx: np.ndarray):
   next_column = np.array([])
   last_f = fx[-1]
   for index, f in np.ndenumerate(fx):
    if f != last_f:
        next_column = np.append(next_column, round((fx[index[0]+1] - f), 2))
   return next_column
print("Sprawdzenie poprawności działania funkcji:")
print(get_next_column(fx))

[ 0.75  4.    8.8  13.8  17.65 19.   16.5 ]


AttributeError: 'numpy.ndarray' object has no attribute 'insert'

# Metoda róznicy wstecznej

In [114]:
fx = np.flip(fx)
def get_next_column_reverse(fx: np.ndarray):
   next_column = np.array([])
   first_f = fx[-1]
   for index, f in np.ndenumerate(fx):
    if f != first_f:
        next_column = np.append(next_column, round((f - fx[index[0]+1] ), 2))
   return next_column
print("Sprawdzenie poprawności działania funkcji:")
print(get_next_column_reverse(fx))

Sprawdzenie poprawności działania funkcji:
[-3.25 -4.8  -5.   -3.85 -1.35  2.5 ]


In [115]:
columns = []

def calculate_cols_reverse(column: np.ndarray):
    global columns
    col = get_next_column_reverse(column)
    print(col)
    columns.append(col)
    if len(np.unique(col)) != 1:
        calculate_cols_reverse(col)
    else:
        return

calculate_cols_reverse(fx)


[-3.25 -4.8  -5.   -3.85 -1.35  2.5 ]
[ 1.55  0.2  -1.15 -2.5  -3.85]
[1.35 1.35 1.35 1.35]


In [116]:
for i in range(len(columns)):
    print(f'Column {i+1}: {columns[i]}')

Column 1: [-3.25 -4.8  -5.   -3.85 -1.35  2.5 ]
Column 2: [ 1.55  0.2  -1.15 -2.5  -3.85]
Column 3: [1.35 1.35 1.35 1.35]


In [117]:
for column in columns:
    print(column[3])

-3.85
-2.5
1.35
